# Лабораторная работа №5
## ----------------------------------------------------------------------------------------------------------------------------------
## Задание:
1 . Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
#### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt


#### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('cybersecurity_attacks.csv')  # Замени на путь к файлу
print(Dataset.head())


             Timestamp Source IP Address Destination IP Address  Source Port  \
0  2023-05-30 06:33:58     103.216.15.12           84.9.164.252        31225   
1  2020-08-26 07:08:30    78.199.217.198         66.191.137.154        17245   
2  2022-11-13 08:23:25      63.79.210.48          198.219.82.17        16811   
3  2023-07-02 10:38:46     163.42.196.10        101.228.192.255        20018   
4  2023-07-16 13:11:07     71.166.185.76        189.243.174.238         6131   

   Destination Port Protocol  Packet Length Packet Type Traffic Type  \
0             17616     ICMP            503        Data         HTTP   
1             48166     ICMP           1174        Data         HTTP   
2             53600      UDP            306     Control         HTTP   
3             32534      UDP            385        Data         HTTP   
4             26646      TCP           1462        Data          DNS   

                                        Payload Data  ... Action Taken  \
0  Qui natus

### Создание искомого столбца

In [3]:
label_encoder = LabelEncoder()
Dataset['Attack Type Encoded'] = label_encoder.fit_transform(Dataset['Attack Type'])
print(Dataset[['Attack Type', 'Attack Type Encoded']].head(10))

  Attack Type  Attack Type Encoded
0     Malware                    2
1     Malware                    2
2        DDoS                    0
3     Malware                    2
4        DDoS                    0
5     Malware                    2
6        DDoS                    0
7   Intrusion                    1
8   Intrusion                    1
9     Malware                    2


### Нормализация данных

In [4]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)

# Заполнение пустых значений средними значениями
columns_to_fill = ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)

print(Dataset)


       Source Port  Destination Port  Packet Length  Anomaly Scores  \
0            31225             17616            503           28.67   
1            17245             48166           1174           51.50   
2            16811             53600            306           87.42   
3            20018             32534            385           15.79   
4             6131             26646           1462            0.52   
...            ...               ...            ...             ...   
39995        31005              6764           1428           39.28   
39996         2553             28091           1184           27.25   
39997        22505             25152           1043           31.01   
39998        20013              2703            483           97.85   
39999        50137             55575           1175           34.63   

       Attack Type Encoded  
0                        2  
1                        2  
2                        0  
3                        2  
4 

### Поиск лучших метрик

In [5]:
X = Dataset.drop(columns=['Attack Type Encoded'])
y = Dataset['Attack Type Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Для решающего дерева
decision_tree = DecisionTreeClassifier()
dt_params = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_grid_search = GridSearchCV(decision_tree, dt_params, cv=5)
dt_grid_search.fit(X_train, y_train)

y_pred_dt = dt_grid_search.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree - Лучшее значение параметра: {dt_grid_search.best_params_}, Точность на тесте: {accuracy_dt}")

# Для случайного леса
random_forest = RandomForestClassifier()
rf_params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(random_forest, rf_params, cv=5)
rf_grid_search.fit(X_train, y_train)

y_pred_rf = rf_grid_search.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest - Лучшие значения параметров: {rf_grid_search.best_params_}, Точность на тесте: {accuracy_rf}")


Decision Tree - Лучшее значение параметра: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5}, Точность на тесте: 0.3265
Random Forest - Лучшие значения параметров: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}, Точность на тесте: 0.326625


### Обучение на лучших параметрах

In [6]:
from sklearn.metrics import classification_report

# Для решающего дерева
best_dt_params = {'max_depth': 20}
best_decision_tree = DecisionTreeClassifier(**best_dt_params)
best_decision_tree.fit(X_train, y_train)

y_pred_best_dt = best_decision_tree.predict(X_test)
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)

print(f"Decision Tree с лучшими параметрами - Точность на тесте: {accuracy_best_dt}")
print("Classification Report для Decision Tree:")
print(classification_report(y_test, y_pred_best_dt))

# Для случайного леса
best_rf_params = {'max_depth': None, 'n_estimators': 50}
best_random_forest = RandomForestClassifier(**best_rf_params)
best_random_forest.fit(X_train, y_train)

y_pred_best_rf = best_random_forest.predict(X_test)
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)

print(f"Random Forest с лучшими параметрами - Точность на тесте: {accuracy_best_rf}")
print("Classification Report для Random Forest:")
print(classification_report(y_test, y_pred_best_rf))


Decision Tree с лучшими параметрами - Точность на тесте: 0.3355625
Classification Report для Decision Tree:
              precision    recall  f1-score   support

           0       0.33      0.40      0.36      5306
           1       0.34      0.35      0.34      5416
           2       0.35      0.26      0.30      5278

    accuracy                           0.34     16000
   macro avg       0.34      0.34      0.33     16000
weighted avg       0.34      0.34      0.33     16000

Random Forest с лучшими параметрами - Точность на тесте: 0.3345625
Classification Report для Random Forest:
              precision    recall  f1-score   support

           0       0.33      0.38      0.35      5306
           1       0.34      0.31      0.32      5416
           2       0.33      0.32      0.32      5278

    accuracy                           0.33     16000
   macro avg       0.33      0.33      0.33     16000
weighted avg       0.33      0.33      0.33     16000

